In [ ]:
#https://www.kaggle.com/insaff/xgboost/code

In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")


In [3]:
test.head()

,ID,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,0,1.375465e+00,11.361141,C,4.200778,6.57700,2.081784,1.784386,0.011094,9.523810,...,7.619048,1.815241,1.112270e-07,AF,1.292368,3.903345,1.485925,0,2.333334,1.428572e+00
1,1,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,I,NaN,NaN,NaN,0,NaN,NaN
2,2,-4.903407e-07,8.201529,C,4.544371,6.55010,1.558442,2.467532,0.007164,7.142858,...,5.714286,1.970928,1.412265e-02,AV,1.128724,5.844156,1.475892,0,1.263157,-6.380022e-07
3,7,2.661870e+00,3.041241,C,1.657216,9.77308,2.078337,1.430855,1.252157,7.959596,...,4.404040,8.163614,1.100329e+00,B,1.988688,1.558753,2.448814,0,5.385474,1.493777e+00
4,10,1.252822e+00,11.283352,C,4.638388,8.52051,2.302484,3.510159,0.074263,7.612904,...,6.580644,1.325654,2.584588e-01,A,1.863796,2.666478,2.374275,0,0.681672,2.264151e+00


In [4]:
train.head()

,ID,target,v1,v2,v3,v4,v5,v6,v7,v8,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,3,1,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,...,8.000000,1.989780,0.035754,AU,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,4,1,NaN,NaN,C,NaN,9.191265,NaN,NaN,2.301630,...,NaN,NaN,0.598896,AF,NaN,NaN,1.957825,0,NaN,NaN
2,5,1,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,...,9.333333,2.477596,0.013452,AE,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,6,1,0.797415,8.304757,C,4.225930,11.627438,2.097700,1.987549,0.171947,...,7.018256,1.812795,0.002267,CJ,1.415230,2.954381,1.990847,1,1.677108,1.034483
4,8,1,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Z,NaN,NaN,NaN,0,NaN,NaN


In [5]:
id_test=test['ID'].values
test=test.drop(['ID'],axis=1)

In [6]:
test = test.drop(['v8','v23','v25','v31','v36','v37','v46','v51','v53','v54','v63','v73','v75','v79','v81','v82','v89','v92','v95','v105','v107','v108','v109','v110','v116','v117','v118','v119','v123','v124','v128'],axis=1)
target = train['target'].values
train = train.drop(['ID'],axis=1)
#train = train.drop(['target'],axis=1)
train = train.drop(['target','v8','v23','v25','v31','v36','v37','v46','v51','v53','v54','v63','v73','v75','v79','v81','v82','v89','v92','v95','v105','v107','v108','v109','v110','v116','v117','v118','v119','v123','v124','v128'],axis=1)

In [7]:
shapeTrain = train.shape[0]
shapeTest = test.shape[0]
train = train.append(test)

In [8]:
train.shape

(228714, 100)

In [9]:
from sklearn import preprocessing 
for f in train.columns: 
    if train[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder() 
        lbl.fit(list(train[f].values)) 
        train[f] = lbl.transform(list(train[f].values))

In [10]:
test = train[shapeTrain:shapeTrain+shapeTest]
train = train[0:shapeTrain]

In [ ]:
#xgboost will handle NA values


In [11]:
train=np.array(train) 
test=np.array(test) 
train = train.astype(float) 
test = test.astype(float)

In [12]:
test

array([[ 1.37546546e+00,  1.13611412e+01,  2.00000000e+00, ...,
         0.00000000e+00,  2.33333377e+00,  1.42857205e+00],
       [            nan,             nan,  2.00000000e+00, ...,
         0.00000000e+00,             nan,             nan],
       [-4.90340722e-07,  8.20152939e+00,  2.00000000e+00, ...,
         0.00000000e+00,  1.26315718e+00, -6.38002238e-07],
       ...,
       [ 1.52373963e+00,  6.57159665e+00,  2.00000000e+00, ...,
         0.00000000e+00,  1.81659428e+00,  1.63461582e+00],
       [            nan,             nan,  2.00000000e+00, ...,
         0.00000000e+00,             nan,             nan],
       [ 1.63355440e+00,  7.52058320e+00,  2.00000000e+00, ...,
         0.00000000e+00,  1.59999966e+00,  1.99999984e+00]])

In [13]:
X_fit, X_eval, y_fit, y_eval= train_test_split(
    train, target, test_size=0.15, random_state=1
)


In [14]:
xgtrain = xgb.DMatrix(X_fit, y_fit)
xgtest = xgb.DMatrix(test)

In [15]:
clf = xgb.XGBClassifier(missing=np.nan, max_depth=7, 
                        n_estimators=300, learning_rate=0.05, 
                        subsample=1, colsample_bytree=0.9, seed=2100,objective= 'binary:logistic')

In [16]:
clf.fit(X_fit, y_fit, early_stopping_rounds=35,  eval_metric="logloss", eval_set=[(X_eval, y_eval)])


[0]	validation_0-logloss:0.67446
Will train until validation_0-logloss hasn't improved in 35 rounds.
[1]	validation_0-logloss:0.657663
[2]	validation_0-logloss:0.6431
[3]	validation_0-logloss:0.629091
[4]	validation_0-logloss:0.616421
[5]	validation_0-logloss:0.604759
[6]	validation_0-logloss:0.59404
[7]	validation_0-logloss:0.584764
[8]	validation_0-logloss:0.575789
[9]	validation_0-logloss:0.568472
[10]	validation_0-logloss:0.560756
[11]	validation_0-logloss:0.553712
[12]	validation_0-logloss:0.547709
[13]	validation_0-logloss:0.542024
[14]	validation_0-logloss:0.536475
[15]	validation_0-logloss:0.531325
[16]	validation_0-logloss:0.527242
[17]	validation_0-logloss:0.522784
[18]	validation_0-logloss:0.518668
[19]	validation_0-logloss:0.514881
[20]	validation_0-logloss:0.51131
[21]	validation_0-logloss:0.508324
[22]	validation_0-logloss:0.505238
[23]	validation_0-logloss:0.502385
[24]	validation_0-logloss:0.499736
[25]	validation_0-logloss:0.497272
[26]	validation_0-logloss:0.495041
[2

[230]	validation_0-logloss:0.461565
[231]	validation_0-logloss:0.461587
[232]	validation_0-logloss:0.4616
[233]	validation_0-logloss:0.461617
[234]	validation_0-logloss:0.461526
[235]	validation_0-logloss:0.46153
[236]	validation_0-logloss:0.46152
[237]	validation_0-logloss:0.461522
[238]	validation_0-logloss:0.461529
[239]	validation_0-logloss:0.461563
[240]	validation_0-logloss:0.461537
[241]	validation_0-logloss:0.461531
[242]	validation_0-logloss:0.461403
[243]	validation_0-logloss:0.461388
[244]	validation_0-logloss:0.461401
[245]	validation_0-logloss:0.461407
[246]	validation_0-logloss:0.461433
[247]	validation_0-logloss:0.461436
[248]	validation_0-logloss:0.461447
[249]	validation_0-logloss:0.461445
[250]	validation_0-logloss:0.46143
[251]	validation_0-logloss:0.461462
[252]	validation_0-logloss:0.46147
[253]	validation_0-logloss:0.461472
[254]	validation_0-logloss:0.461421
[255]	validation_0-logloss:0.461451
[256]	validation_0-logloss:0.461406
[257]	validation_0-logloss:0.46143

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.9, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=2100,
       silent=True, subsample=1)

In [17]:
from  sklearn.metrics import log_loss
log_train = log_loss(y_fit, clf.predict_proba(X_fit)[:,1])
log_valid = log_loss(y_eval, clf.predict_proba(X_eval)[:,1])

In [18]:
print('\n-----------------------')
print('  logloss train: %.5f'%log_train)
print('  logloss valid: %.5f'%log_valid)
print('-----------------------')


-----------------------
  logloss train: 0.40307
  logloss valid: 0.46119
-----------------------


In [19]:
print('\nModel parameters...')
print(clf.get_params())


Model parameters...
{'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bytree': 0.9, 'gamma': 0, 'learning_rate': 0.05, 'max_delta_step': 0, 'max_depth': 7, 'min_child_weight': 1, 'missing': None, 'n_estimators': 300, 'n_jobs': 1, 'nthread': None, 'objective': 'binary:logistic', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'scale_pos_weight': 1, 'seed': 2100, 'silent': True, 'subsample': 1}


In [20]:
y_pred= clf.predict_proba(test,ntree_limit=clf.best_ntree_limit)[:,1]
submission = pd.DataFrame({"ID":id_test, "PredictedProb":y_pred})
submission.to_csv("submission3.csv", index=False)